# Whisper Plugin

> Plugin implementation for OpenAI Whisper transcription

In [ ]:
#| default_exp plugin

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
import logging
from pathlib import Path
from typing import Dict, Any, Optional, List, Union
import tempfile
import warnings

import numpy as np
import torch
import soundfile as sf

try:
    import whisper
    from whisper import load_model
    from whisper import transcribe
    from cjm_ffmpeg_utils.core import FFMPEG_AVAILABLE
    WHISPER_AVAILABLE = True and FFMPEG_AVAILABLE
except ImportError:
    WHISPER_AVAILABLE = False
    
from cjm_transcription_plugin_system.plugin_interface import TranscriptionPlugin
from cjm_transcription_plugin_system.core import AudioData, TranscriptionResult

In [ ]:
#| export
class WhisperLocalPlugin(TranscriptionPlugin):
    """OpenAI Whisper transcription plugin."""
    
    def __init__(self):
        """Initialize the Whisper plugin with default configuration."""
        self.logger = logging.getLogger(f"{__name__}.{type(self).__name__}")
        self.config = {}
        self.model = None
        self.device = None
        self.model_dir = None
    
    @property
    def name(
        self
    ) -> str: # the plugin name identifier
        """Get the plugin name identifier."""
        return "whisper_local"
    
    @property
    def version(
        self
    ) -> str: # the plugin version string
        """Get the plugin version string."""
        return "1.0.0"
    
    @property
    def supported_formats(
        self
    ) -> List[str]: # list of supported audio file formats
        """Get the list of supported audio file formats."""
        return ["wav", "mp3", "flac", "m4a", "ogg", "webm", "mp4", "avi", "mov"]

    @staticmethod
    def get_config_schema(
    ) -> Dict[str, Any]: # the configuration schema dictionary
        """Return configuration schema for Whisper."""
        return {
            "$schema": "http://json-schema.org/draft-07/schema#",
            "type": "object",
            "title": "Whisper Configuration",
            "properties": {
                "model": {
                    "type": "string",
                    "enum": ["tiny", "tiny.en", "base", "base.en", "small", "small.en", 
                            "medium", "medium.en", "large", "large-v1", "large-v2", "large-v3"],
                    "default": "base",
                    "description": "Whisper model size. Larger models are more accurate but slower."
                },
                "device": {
                    "type": "string",
                    "enum": ["auto", "cpu", "cuda"],
                    "default": "auto",
                    "description": "Device for inference (auto will use CUDA if available)"
                },
                "language": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Language code (e.g., 'en', 'es', 'fr') or null for auto-detection"
                },
                "task": {
                    "type": "string",
                    "enum": ["transcribe", "translate"],
                    "default": "transcribe",
                    "description": "Task to perform (transcribe or translate to English)"
                },
                "temperature": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.0,
                    "description": "Sampling temperature (0 for deterministic)"
                },
                "temperature_increment_on_fallback": {
                    "type": ["number", "null"],
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.2,
                    "description": "Temperature increment when falling back"
                },
                "beam_size": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 10,
                    "default": 5,
                    "description": "Beam search width"
                },
                "best_of": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 10,
                    "default": 5,
                    "description": "Number of candidates when sampling"
                },
                "patience": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 2.0,
                    "default": 1.0,
                    "description": "Beam search patience factor"
                },
                "length_penalty": {
                    "type": ["number", "null"],
                    "default": None,
                    "description": "Exponential length penalty"
                },
                "suppress_tokens": {
                    "type": "string",
                    "default": "-1",
                    "description": "Tokens to suppress ('-1' for default)"
                },
                "initial_prompt": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Optional initial prompt"
                },
                "condition_on_previous_text": {
                    "type": "boolean",
                    "default": False,
                    "description": "Condition on previous text"
                },
                "fp16": {
                    "type": "boolean",
                    "default": True,
                    "description": "Use FP16 (half precision) for faster inference"
                },
                "compression_ratio_threshold": {
                    "type": "number",
                    "minimum": 1.0,
                    "maximum": 10.0,
                    "default": 2.4,
                    "description": "Threshold for repetition detection"
                },
                "logprob_threshold": {
                    "type": "number",
                    "default": -1.0,
                    "description": "Average log probability threshold"
                },
                "no_speech_threshold": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.6,
                    "description": "Threshold for silence detection"
                },
                "word_timestamps": {
                    "type": "boolean",
                    "default": False,
                    "description": "Extract word-level timestamps"
                },
                "prepend_punctuations": {
                    "type": "string",
                    "default": "\"'“¿([{-",
                    "description": "Punctuations to merge with next word"
                },
                "append_punctuations": {
                    "type": "string",
                    "default": "\"'.。,，!！?？:：”)]}、",
                    "description": "Punctuations to merge with previous word"
                },
                "threads": {
                    "type": "integer",
                    "minimum": 0,
                    "default": 0,
                    "description": "Number of threads (0 for default)"
                },
                "model_dir": {
                    "type": ["string", "null"],
                    "default": None,
                    "description": "Directory to save/load models"
                },
                "compile_model": {
                    "type": "boolean",
                    "default": False,
                    "description": "Use torch.compile for potential speedup (requires PyTorch 2.0+)"
                }
            },
            "required": ["model"],
            "additionalProperties": False
        }
    
    def get_current_config(
        self
    ) -> Dict[str, Any]: # the current configuration dictionary
        """Return current configuration."""
        defaults = self.get_config_defaults()
        return {**defaults, **self.config}
    
    def initialize(
        self,
        config: Optional[Dict[str, Any]] = None # configuration dictionary to initialize the plugin
    ) -> None:
        """Initialize the plugin with configuration."""
        if config:
            is_valid, error = self.validate_config(config)
            if not is_valid:
                raise ValueError(f"Invalid configuration: {error}")
        
        # Merge with defaults
        defaults = self.get_config_defaults()
        self.config = {**defaults, **(config or {})}
        
        # Set device
        if self.config["device"] == "auto":
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = self.config["device"]
        
        # Set model directory
        self.model_dir = self.config.get("model_dir")
        
        self.logger.info(f"Initialized Whisper plugin with model '{self.config['model']}' on device '{self.device}'")
    
    def _load_model(
        self
    ) -> None:
        """Load the Whisper model (lazy loading)."""
        if self.model is None:
            try:
                self.logger.info(f"Loading Whisper model: {self.config['model']}")
                self.model = load_model(
                    self.config["model"], 
                    device=self.device,
                    download_root=self.model_dir
                )
                
                # Optionally compile the model (PyTorch 2.0+)
                if self.config.get("compile_model", False) and hasattr(torch, 'compile'):
                    self.model = torch.compile(self.model)
                    self.logger.info("Model compiled with torch.compile")
                    
                self.logger.info("Local Whisper model loaded successfully")
            except Exception as e:
                raise RuntimeError(f"Failed to load Whisper model: {e}")
    
    def _prepare_audio(
        self,
        audio: Union[AudioData, str, Path] # audio data, file path, or Path object to prepare
    ) -> str: # path to the prepared audio file
        """Prepare audio for Whisper processing."""
        if isinstance(audio, (str, Path)):
            # Already a file path
            return str(audio)
        
        elif isinstance(audio, AudioData):
            # Save AudioData to temporary file
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
                # Ensure audio is in the correct format
                audio_array = audio.samples
                
                # If stereo, convert to mono
                if audio_array.ndim > 1:
                    audio_array = audio_array.mean(axis=1)
                
                # Ensure float32 and normalized
                if audio_array.dtype != np.float32:
                    audio_array = audio_array.astype(np.float32)
                
                # Normalize if needed
                if audio_array.max() > 1.0:
                    audio_array = audio_array / np.abs(audio_array).max()
                
                # Save to file
                sf.write(tmp_file.name, audio_array, audio.sample_rate)
                return tmp_file.name
        else:
            raise ValueError(f"Unsupported audio input type: {type(audio)}")
    
    def execute(
        self,
        audio: Union[AudioData, str, Path], # audio data or path to audio file to transcribe
        **kwargs # additional arguments to override config
    ) -> TranscriptionResult: # transcription result with text and metadata
        """Transcribe audio using Whisper."""
        # Load model if not already loaded
        self._load_model()
        
        # Prepare audio file
        audio_path = self._prepare_audio(audio)
        temp_file_created = not isinstance(audio, (str, Path))
        
        try:
            # Merge runtime kwargs with config
            exec_config = {**self.config, **kwargs}
            
            # Prepare Whisper arguments
            whisper_args = {
                "verbose": False,
                "task": exec_config["task"],
                "language": exec_config["language"],
                "beam_size": exec_config["beam_size"],
                "best_of": exec_config["best_of"],
                "patience": exec_config["patience"],
                "length_penalty": exec_config["length_penalty"],
                "suppress_tokens": exec_config["suppress_tokens"],
                "initial_prompt": exec_config["initial_prompt"],
                "condition_on_previous_text": exec_config["condition_on_previous_text"],
                "fp16": exec_config["fp16"] and self.device == "cuda",
                "compression_ratio_threshold": exec_config["compression_ratio_threshold"],
                "logprob_threshold": exec_config["logprob_threshold"],
                "no_speech_threshold": exec_config["no_speech_threshold"],
                "word_timestamps": exec_config["word_timestamps"],
                "prepend_punctuations": exec_config["prepend_punctuations"],
                "append_punctuations": exec_config["append_punctuations"],
            }
            
            # Handle temperature settings
            temperature = exec_config["temperature"]
            temp_increment = exec_config["temperature_increment_on_fallback"]
            if temp_increment is not None and temp_increment > 0:
                temperature = tuple(np.arange(temperature, 1.0 + 1e-6, temp_increment))
            else:
                temperature = [temperature]
            
            # Set number of threads if specified
            if exec_config["threads"] > 0:
                torch.set_num_threads(exec_config["threads"])
            
            # Perform transcription
            self.logger.info(f"Transcribing audio with Whisper {exec_config['model']}")
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")  # Suppress Whisper warnings
                result = transcribe(
                    self.model,
                    audio_path,
                    temperature=temperature,
                    **whisper_args
                )
            
            # Process segments
            segments = []
            for segment in result.get("segments", []):
                segment_data = {
                    "start": segment["start"],
                    "end": segment["end"],
                    "text": segment["text"].strip()
                }
                
                # Add word timestamps if available
                if "words" in segment and exec_config["word_timestamps"]:
                    segment_data["words"] = [
                        {
                            "word": word["word"],
                            "start": word["start"],
                            "end": word["end"],
                            "probability": word.get("probability")
                        }
                        for word in segment["words"]
                    ]
                
                segments.append(segment_data)
            
            # Create transcription result
            transcription_result = TranscriptionResult(
                text=result["text"].strip(),
                confidence=None,  # Whisper doesn't provide overall confidence
                segments=segments if segments else None,
                metadata={
                    "model": exec_config["model"],
                    "language": result.get("language", exec_config["language"]),
                    "task": exec_config["task"],
                    "device": self.device,
                    "duration": result.get("duration"),
                }
            )
            
            self.logger.info(f"Transcription completed: {len(result['text'].split())} words")
            return transcription_result
            
        finally:
            # Clean up temporary file if created
            if temp_file_created:
                try:
                    Path(audio_path).unlink()
                except Exception:
                    pass
    
    def is_available(
        self
    ) -> bool: # True if Whisper and its dependencies are available
        """Check if Whisper is available."""
        return WHISPER_AVAILABLE
    
    def cleanup(
        self
    ) -> None:
        """Clean up resources."""
        if self.model is not None:
            self.logger.info("Unloading Whisper model")
            self.model = None
            
            # Clear GPU cache if using CUDA
            if self.device == "cuda" and torch.cuda.is_available():
                torch.cuda.empty_cache()
                
            self.logger.info("Cleanup completed")

## Testing the Plugin

In [ ]:
# Test basic functionality
plugin = WhisperLocalPlugin()

# Check availability
print(f"Whisper available: {plugin.is_available()}")
print(f"Plugin name: {plugin.name}")
print(f"Plugin version: {plugin.version}")
print(f"Supported formats: {plugin.supported_formats}")

Whisper available: True
Plugin name: whisper_local
Plugin version: 1.0.0
Supported formats: ['wav', 'mp3', 'flac', 'm4a', 'ogg', 'webm', 'mp4', 'avi', 'mov']


In [ ]:
# Test configuration schema
schema = plugin.get_config_schema()
print("Available models:")
for model in schema["properties"]["model"]["enum"]:
    print(f"  - {model}")

Available models:
  - tiny
  - tiny.en
  - base
  - base.en
  - small
  - small.en
  - medium
  - medium.en
  - large
  - large-v1
  - large-v2
  - large-v3


In [ ]:
# Test configuration validation
test_configs = [
    ({"model": "tiny"}, "Valid config"),
    ({"model": "invalid"}, "Invalid model"),
    ({"temperature": 1.5}, "Temperature out of range"),
]

for config, description in test_configs:
    is_valid, error = plugin.validate_config(config)
    print(f"{description}: Valid={is_valid}")
    if error:
        print(f"  Error: {error[:100]}")

Valid config: Valid=True
Invalid model: Valid=False
  Error: 'invalid' is not one of ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'medium', 'mediu
Temperature out of range: Valid=False
  Error: 'model' is a required property

Failed validating 'required' in schema:
    {'$schema': 'http://json


In [ ]:
# Test initialization
plugin.initialize({"model": "tiny", "device": "cpu"})
print(f"Current config: {plugin.get_current_config()['model']}")

Current config: tiny


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()